In [1]:
#####欠損値のある変分ベイズ行列因子分解#####
##ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [2]:
####データの発生####
#np.random.seed(58732)   #シードを設定

##データの設定
k = 15   #基底数
hh = 5000   #ユーザー数
item = 1500   #アイテム数

##パラメータの設定
sigma = 1
A = A_T = np.random.multivariate_normal(np.repeat(0, k), np.identity(k), hh)   #ユーザーの特徴行列
B = B_T = np.random.multivariate_normal(np.repeat(0, k), np.identity(k), item)   #アイテムの特徴行列
beta1 = np.random.beta(8.5, 10.0, hh)   #ユーザー購買確率
beta2 = np.random.beta(5.0, 6.0, item)   #アイテム購買確率

In [3]:
##モデルに基づき応答変数を生成
AB = np.dot(A, B.T)
Y = np.zeros((hh, item))
Z = np.zeros((hh, item))

for j in range(item):
    #評価ベクトルを生成
    y_vec = np.random.normal(AB[:, j], sigma, hh)   #正規分布から評価ベクトルを生成
    
    #欠損を生成
    deficit = np.random.binomial(1, beta1 * beta2[j], hh)
    Z[:, j] = deficit   #欠損を代入
    
    #評価ベクトルを代入
    Y[:, j] = y_vec

In [4]:
##IDと評価ベクトルを設定
N = Z[Z==1].shape[0]
user_id0 = (np.repeat(range(hh), item))
item_id0 = np.tile((np.array(range(item))), hh)

#評価がある要素のみ抽出
index_user = (np.ravel(Z).astype(np.int64)==1).nonzero()
user_id = user_id0[index_user]
item_id = item_id0[index_user]
y_vec = np.ravel(Y)

In [5]:
##評価ベクトルを1～5の間の離散値に収める
score_mu = 3
y0 = np.round(scipy.stats.zscore(y_vec)) + score_mu
y0[y0 < 1] = 1
y0[y0 > 5] = 5
y = y0[index_user]   #欠損のある評価ベクトル
Y = y0.reshape(item, hh)   #評価ベクトルの完全データ

In [6]:
##インデックスを作成
index = np.array(range(y.shape[0]))
index_user = [i for i in range(hh)]
index_item = [j for j in range(item)]
for i in range(hh):
    index_user[i] = index[user_id==(i)]
for j in range(item):
    index_item[j] = index[item_id==(j)]

In [7]:
####変分ベイズ法でパラメータを推定####
##アルゴリズムの設定
LL1 = -1000000000   #対数尤度の初期値
tol = 1
iter = 1
dl = 100

##事前分布の設定
sigma = 1
Ca = np.identity(k)
Cb = np.identity(k)

In [8]:
##初期値の設定
Cov_A = np.zeros((k, k, hh))
Cov_B = np.zeros((k, k, item))
for i in range(hh):
    Cov_A[:, :, i] = np.identity(k)
for j in range(item):
    Cov_B[:, :, j] = np.identity(k)
A = np.random.multivariate_normal(np.repeat(0, k), 0.2 * np.identity(k), hh)
B = np.random.multivariate_normal(np.repeat(0, k), 0.2 * np.identity(k), item)

In [9]:
####変分ベイズ法でパラメータを更新####
while abs(dl) >= tol:   #dlがtol以上なら繰り返す

    ##ユーザー特徴行列のパラメータを更新
    A = np.zeros((hh, k))
    Cov_A = np.zeros((k, k, hh))

    #分散成分を更新
    for i in range(hh):
        index = item_id[index_user[i]]
        Cov_sum = np.sum(Cov_B[:, :, index], axis=2)
        inv_Ca = np.linalg.inv(Ca)
        Cov_A[:, :, i] = np.power(sigma, 2) * (np.linalg.inv((np.dot(B[index, :].T, B[index, :]) + Cov_sum) + np.power(sigma, 2) * inv_Ca))

    #ユーザーごとの特徴ベクトルを更新
    for i in range(hh):
        By = np.sum(np.tile(y[index_user[i]], k).reshape((k, index_user[i].shape[0])).T * B[item_id[index_user[i]], :], axis=0)
        A[i, :] = pow(sigma, -2) * np.dot(Cov_A[:, :, i], By)


    ##アイテムの特徴行列のパラメータを更新
    i = 0
    B = np.zeros((item, k))
    Cov_B = np.zeros((k, k, item))

    #分散成分を更新
    for j in range(item):
        index = user_id[index_item[j]]
        Cov_sum = np.sum(Cov_A[:, :, index], axis=2)
        inv_Cb = np.linalg.inv(Cb)
        Cov_B[:, :, j] = np.power(sigma, 2) * (np.linalg.inv((np.dot(A[index, :].T, A[index, :]) + Cov_sum) + np.power(sigma, 2) * inv_Cb))

    #アイテムごとの特徴ベクトルを更新
    for j in range(item):
        Ay = np.sum(np.tile(y[index_item[j]], k).reshape((k, index_item[j].shape[0])).T * A[user_id[index_item[j]], :], axis=0)
        B[j, :] = pow(sigma, -2) * np.dot(Cov_B[:, :, j], Ay)


    ##ハイパーパラメータを更新
    #パラメータの事前分布を更新
    Ca = np.diag((np.sum(np.power(A, 2), axis=0) + np.diag(np.sum(Cov_A, axis=2))) / hh)
    Cb = np.diag((np.sum(np.power(B, 2), axis=0) + np.diag(np.sum(Cov_B, axis=2))) / item)

    #標準偏差の更新
    score = np.sum(A[user_id, :] * B[item_id, :], axis=1)
    sigma = np.sqrt(np.sum(np.power(y - score, 2)) / y.shape[0])


    ##アルゴリズムの収束判定
    LL = np.sum(scipy.stats.norm.logpdf(y, score, sigma))
    iter = iter + 1
    dl = LL -LL1
    LL1 = LL
    print(LL)

-3185265.05141
-1994332.15962
-1093959.2614
-931877.611456
-926836.787548
-923415.411935
-920768.636119
-918690.470996
-917018.151346
-915632.045862
-914450.360181
-913419.701422
-912506.075608
-911687.891117
-910951.076586
-910285.913158
-909685.099284
-909142.637476
-908653.237593
-908212.026685
-907814.429155
-907456.133382
-907133.095965
-906841.55708
-906578.053964
-906339.427299
-906122.819419
-905925.665247
-905745.677614
-905580.828765
-905429.329622
-905289.608096
-905160.287433
-905040.165271
-904928.193867
-904823.46178
-904725.177121
-904632.652443
-904545.291207
-904462.575777
-904384.056827
-904309.344051
-904238.09805
-904170.023279
-904104.861931
-904042.38865
-903982.405981
-903924.740446
-903869.239176
-903815.767023
-903764.204084
-903714.443581
-903666.390043
-903619.957752
-903575.069415
-903531.655022
-903489.650871
-903448.998732
-903409.645124
-903371.540709
-903334.639752
-903298.899676
-903264.280662
-903230.745319
-903198.258389
-903166.786504
-903136.297963


In [45]:
####完全データに対する二乗誤差####
#評価ベクトルと欠損ベクトルを設定
z_vec = Z.reshape(-1, )
y_vec = Y.reshape(-1, )
mu_vec = np.dot(A, B.T).reshape(-1, )

#観測データの二乗誤差
er_obz = np.sum(np.power(y_vec[z_vec==1] - mu_vec[z_vec==1], 2))
print(er_obz / sum(z_vec))

#欠損データの二乗誤差
er_na = np.sum(np.power(y_vec[z_vec==0] - mu_vec[z_vec==0], 2))
print(er_na / sum(1-z_vec))

#観測スコアと予測スコアの比較
N = z_vec.shape[0]
result_data = pd.DataFrame(np.concatenate((z_vec.reshape((N, 1)), y_vec.reshape((N, 1)), mu_vec.reshape((N, 1))), axis=1),
                          columns=["欠損", "観測スコア", "予測スコア"])
print(result_data)

0.185934204004
0.212755755321
          欠損  観測スコア     予測スコア
0        1.0    2.0  2.701111
1        1.0    3.0  3.353322
2        0.0    4.0  3.995800
3        0.0    2.0  3.051850
4        0.0    3.0  3.143646
5        1.0    2.0  1.933217
6        0.0    4.0  3.894456
7        0.0    3.0  3.009379
8        0.0    4.0  3.959777
9        0.0    3.0  3.192804
10       0.0    3.0  3.068903
11       0.0    2.0  1.954863
12       1.0    3.0  1.804495
13       0.0    5.0  4.209014
14       1.0    4.0  3.821737
15       0.0    4.0  3.407466
16       0.0    2.0  2.578353
17       0.0    4.0  3.591022
18       0.0    2.0  2.499079
19       0.0    4.0  3.633369
20       1.0    4.0  3.235620
21       1.0    3.0  3.053039
22       1.0    5.0  4.706055
23       0.0    2.0  1.703016
24       0.0    3.0  3.573102
25       0.0    2.0  2.906757
26       0.0    5.0  4.717012
27       0.0    4.0  3.512904
28       0.0    4.0  4.275126
29       1.0    2.0  2.417716
...      ...    ...       ...
7499970  0